In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer= LancasterStemmer()

In [2]:
import numpy as np
import tflearn
import tensorflow as tf
import random
import json

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
with open("intents.json") as file:
    data=json.load(file)
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day'],
   'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
   'context_set': ''},
  {'tag': 'age',
   'patterns': ['how old',
    'how old is tim',
    'what is your age',
    'how old are you',
    'age?'],
   'responses': ['I am 18 years old!', '18 years young!'],
   'context_set': ''},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Tim.',
    "I'm Tim!",
    "I'm Tim aka Tech With Tim."],
   'context_set': ''},
  {'tag': 'shop',
   'patterns': ['Id like to buy something',
    'whats on the menu',


In [4]:
words=[]
labels=[]
docs_x=[]
docs_y=[]

In [5]:
nltk.download('punkt')
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
        if intent["tag"] not in labels:
            labels.append(intent["tag"])

[nltk_data] Downloading package punkt to C:\Users\Pasan
[nltk_data]     Devin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
words=[stemmer.stem(w.lower()) for w in words if w != "?"]
words= sorted(list(set(words)))

labels=sorted(labels)

In [7]:
training=[]
output=[]

In [8]:
out_empty=[0 for _ in range(len(labels))]

for x,doc in enumerate(docs_x):
    bag=[]
    wrds=[stemmer.stem(w) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row=out_empty[:]
    output_row[labels.index(docs_y[x])]=1
    
    training.append(bag)
    output.append(output_row)


In [9]:
    training=np.array(training)
output=np.array(output)

In [13]:
# tf.reset_default_graph()

net=tflearn.input_data(shape=[None,len(training[0])])
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,len(output[0]),activation="softmax")
net=tflearn.regression(net)

model=tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
model.fit(training,output, n_epoch=1000,batch_size=8,show_metric=True)
model.save("model.tflearn")

Training Step: 3999  | total loss: 0.01803 | time: 0.009s
| Adam | epoch: 1000 | loss: 0.01803 - acc: 1.0000 -- iter: 24/26
Training Step: 4000  | total loss: 0.01847 | time: 0.013s
| Adam | epoch: 1000 | loss: 0.01847 - acc: 1.0000 -- iter: 26/26
--
INFO:tensorflow:D:\Data_Science\Projects\ChatBot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
